In [70]:
from unidecode import unidecode
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [71]:
def clean_name(name):
    # Enlever les accents
    cleaned_name = unidecode(name)
    # Mettre en minuscules
    cleaned_name = cleaned_name.lower()
    # Supprimer les mots entre parenthèses
    cleaned_name = re.sub(r'\([^)]*\)', '', cleaned_name)
    return cleaned_name

#  Instinct animal 

In [72]:
url = "https://www.instinct-animal.fr/liste-animaux-sauvages-a-z/"

response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')

animal_links = []
for column in soup.find_all('div', class_='et_pb_column_3_4'):
    for link in column.find_all('a', href=True):
        href = link['href']
        animal_links.append(href)

# Création d'un dataframe contenant les informations scrappées
df_animals = pd.DataFrame(animal_links, columns=['AnimalLinks'])
pd.set_option('display.max_colwidth', None)
df_animals = df_animals.drop(df_animals.index[-1])

df_animals



,AnimalLinks
0,https://www.instinct-animal.fr/oiseaux/accenteur-mouchet/
1,https://www.instinct-animal.fr/addax/
2,https://www.instinct-animal.fr/oiseaux/aigle-royal/
3,https://www.instinct-animal.fr/oiseaux/aigrette-garzette/
4,https://www.instinct-animal.fr/reptiles/alligator-amerique/
...,...
413,https://www.instinct-animal.fr/zebre-de-burchell/
414,https://www.instinct-animal.fr/zebre-de-grevy/
415,https://www.instinct-animal.fr/zebre-de-montagne/
416,https://www.instinct-animal.fr/zibeline/


In [73]:
soup

<!DOCTYPE html>

<html lang="fr-FR">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="https://www.instinct-animal.fr/xmlrpc.php" rel="pingback"/>
<script type="text/javascript">
		document.documentElement.className = 'js';
	</script>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/><style id="et-divi-open-sans-inline-css">/* Original: https://fonts.googleapis.com/css?family=Open+Sans:300italic,400italic,600italic,700italic,800italic,400,300,600,700,800&#038;subset=latin,latin-ext&#038;display=swap *//* User Agent: Mozilla/5.0 (Unknown; Linux x86_64) AppleWebKit/538.1 (KHTML, like Gecko) Safari/538.1 Daum/4.1 */@font-face {font-family: 'Open Sans';font-style: italic;font-weight: 300;font-stretch: normal;font-display: swap;src: url(https://fonts.gstatic.com/s/opensans/v40/memQYaGs126MiZpBA-UFUIcVXSCEkx2cmqvXlWq8tWZ0Pw86hd0Rk5hkWV4exQ.ttf) format('truetype');}@font-face {font-family: 'Open Sans';font-style: italic;fon

In [74]:
soup = BeautifulSoup(html_content, 'html.parser')

animal_names = soup.find_all('h3')
animal_names_list = []
for name in animal_names:
    animal_names_list.append(name.get_text())
df_noms = pd.DataFrame({'name': animal_names_list})


In [75]:
df_noms

,name
0,Animaux commençant par la lettre A
1,Accenteur mouchet
2,Addax
3,Aigle royal
4,Aigrette garzette
...,...
440,Zèbre de Burchell
441,Zèbre de Grevy
442,Zèbre de montagne
443,Zibeline


In [76]:
mask = ~df_noms['name'].str.startswith('Animaux commençant')
df_noms = df_noms[mask]

In [77]:
df_noms['name']=df_noms['name'].apply(clean_name)

C:\Users\valen\AppData\Local\Temp\ipykernel_26912\872394848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_noms['name']=df_noms['name'].apply(clean_name)


In [78]:
df_noms

,name
1,accenteur mouchet
2,addax
3,aigle royal
4,aigrette garzette
5,alligator americain
...,...
440,zebre de burchell
441,zebre de grevy
442,zebre de montagne
443,zibeline


In [79]:
liste_noms = df_noms['name'].tolist()

In [80]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time


animal_info_list = []
animal_links_list = []

# Iteration dans la colonne AnimalLinks
for index, row in df_animals.iterrows():
    animal_link = row['AnimalLinks']
    animal_links_list.append(animal_link)

session = requests.Session()

# Iterer sur chaque lien dans la liste des liens
for animal_link in animal_links_list:
    try:
        time.sleep(1)
        response = session.get(animal_link)
        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')

        ul_tag = soup.find('div', class_="et_pb_with_border et_pb_module et_pb_text et_pb_text_4 et_pb_text_align_left et_pb_bg_layout_light").find('ul')

        animal_info = {}

        for li_tag in ul_tag.find_all('li'):
            li_text = li_tag.get_text(strip=True)
            key, value = map(lambda x: x.strip(), li_text.split(':', 1))

            # Ajouter l'information à animal_info
            animal_info[key] = value

        div_tag = soup.find('div', class_="et_pb_with_border et_pb_row_inner et_pb_row_inner_1")
        strong_tag = div_tag.find(['strong', 'b'])
        animal_name = strong_tag.get_text(strip=True)
        # Ajouter le nom à animal_info
        animal_info['Nom'] = animal_name
        animal_info_list.append(animal_info.copy())

    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête : {e}")

# Création d'un nouveau dataframe avec les informations ajoutées
df_animal_info = pd.DataFrame(animal_info_list)

df_animal_info


Une erreur s'est produite lors de la requête : ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Une erreur s'est produite lors de la requête : ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))
Une erreur s'est produite lors de la requête : ('Connection aborted.', ConnectionResetError(10054, 'Une connexion existante a dû être fermée par l’hôte distant', None, 10054, None))


,Règne,Embranchement,Classe,Ordre,Famille,Genre,Espèce,Nom latin,Nom
0,Animal,Chordé vertébré,Oiseau,Passériforme,Prunellidé,Prunella,modularis,Prunella modularis,accenteur mouchet
1,Animal,Chordé vertébré,Mammifère placentaire,Artiodactyle ruminant,Bovidé hippotraginé,Addax,nasomaculatus,Addax nasomaculatus,addax
2,Animal,Chordé vertébré,Oiseau,Accipitriforme,Accipitridé,Aquila,chrysaetos,Aquila chrysaetos,aigle royal
3,Animal,Chordé vertébré,Oiseau,Pélécaniforme,Ardéidé,Egretta,garzetta,Egretta garzetta,aigrette garzette
4,Animal,Chordé vertébré,Reptile,Crocodilien,Alligatoridé alligatoriné,Alligator,mississippiensis,Alligator mississippiensis,alligator d’Amérique
...,...,...,...,...,...,...,...,...,...
410,Animal,Chordé vertébré,Mammifère placentaire,Périssodactyle,Equidé,Equus,quagga,Equus quagga,zèbre de Burchell
411,Animal,Chordé vertébré,Mammifère placentaire,Périssodactyle,Equidé,Equus,grevyi,Equus grevyi,zèbre de Grevy
412,Animal,Chordé vertébré,Mammifère placentaire,Périssodactyle,Equidé,Equus,zebra,Equus zebra,zèbre de montagne
413,Animal,Chordé vertébré,Mammifère placentaire,Carnivore caniforme,Mustélidé mustéliné,Martes,zibellina,Martes zibellina,zibeline


In [81]:
df_animal_info["Règne"]=df_animal_info["Règne"].apply(clean_name)
df_animal_info["Embranchement"]=df_animal_info["Embranchement"].apply(clean_name)
df_animal_info["Classe"]=df_animal_info["Classe"].apply(clean_name)
df_animal_info["Ordre"]=df_animal_info["Ordre"].apply(clean_name)
df_animal_info["Famille"]=df_animal_info["Famille"].apply(clean_name)
df_animal_info["Genre"]=df_animal_info["Genre"].apply(clean_name)
df_animal_info["Espèce"]=df_animal_info["Espèce"].apply(clean_name)
df_animal_info["Nom"]=df_animal_info["Nom"].apply(clean_name)
df_animal_info["Nom latin"]=df_animal_info["Nom latin"].apply(clean_name)

In [82]:
df_animal_info

,Règne,Embranchement,Classe,Ordre,Famille,Genre,Espèce,Nom latin,Nom
0,animal,chorde vertebre,oiseau,passeriforme,prunellide,prunella,modularis,prunella modularis,accenteur mouchet
1,animal,chorde vertebre,mammifere placentaire,artiodactyle ruminant,bovide hippotragine,addax,nasomaculatus,addax nasomaculatus,addax
2,animal,chorde vertebre,oiseau,accipitriforme,accipitride,aquila,chrysaetos,aquila chrysaetos,aigle royal
3,animal,chorde vertebre,oiseau,pelecaniforme,ardeide,egretta,garzetta,egretta garzetta,aigrette garzette
4,animal,chorde vertebre,reptile,crocodilien,alligatoride alligatorine,alligator,mississippiensis,alligator mississippiensis,alligator d'amerique
...,...,...,...,...,...,...,...,...,...
410,animal,chorde vertebre,mammifere placentaire,perissodactyle,equide,equus,quagga,equus quagga,zebre de burchell
411,animal,chorde vertebre,mammifere placentaire,perissodactyle,equide,equus,grevyi,equus grevyi,zebre de grevy
412,animal,chorde vertebre,mammifere placentaire,perissodactyle,equide,equus,zebra,equus zebra,zebre de montagne
413,animal,chorde vertebre,mammifere placentaire,carnivore caniforme,mustelide musteline,martes,zibellina,martes zibellina,zibeline


In [83]:
df_animal_info.columns

Index(['Règne', 'Embranchement', 'Classe', 'Ordre', 'Famille', 'Genre',
       'Espèce', 'Nom latin', 'Nom'],
      dtype='object')

In [84]:
df_animal_info = df_animal_info.rename(columns={'Nom': 'name'})

In [136]:
df_animal_info.columns

Index(['Règne', 'Embranchement', 'Classe', 'Ordre', 'Famille', 'Genre',
       'Espèce', 'Nom latin', 'name'],
      dtype='object')

# Monde Animal 

In [85]:
#contexte , hypotheses etc, donnes scrappes, difficultes rencontres et comment vous avez valorise vos données. 

Difficulté rencontrée : Nom different selon le site, se base uniqueemenbt sur le nom latin qui equivaut au nom scientifique. Les deux premieres sites sont ders sites simplement developes donc plus difficile a obtenir des informations precises sur des pages simples. Sites pas connu donc listes d'animaux restreintes.  On voulait partir sur les etres vivants en general mais l'operationel et le temps nous a limite des sites d'animaux. Necessite de scrapper plus de colonnes donc plus d'informations.

In [167]:
url = "https://www.monde-animal.fr/liste-animaux-a-z/"
response = requests.get(url)
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')

In [168]:
sections = soup.find_all('section', class_='elementor-section')

animal_data = []
animal_links = []


for section in sections:
    links = section.find_all('a')
    for link in links:
        animal_name = link.get_text(strip=True)
        animal_link = link['href']
        animal_data.append(animal_name)
        animal_links.append(animal_link)

df_animals_bis_bis = pd.DataFrame({"name":animal_data,'Links':animal_links})
df_animals_bis_bis

,name,Links
0,,https://www.monde-animal.fr/
1,Accueil,https://www.monde-animal.fr/
2,Liste des animaux de A à Z,https://www.monde-animal.fr/liste-animaux-a-z/
3,Fiches Animaux,https://www.monde-animal.fr/fiches-animaux/
4,Blog,https://www.monde-animal.fr/blog/
...,...,...
386,Wapiti,https://www.monde-animal.fr/fiche_animaux/wapiti/
387,Wombat commun,https://www.monde-animal.fr/fiche_animaux/wombat-commun/
388,Zèbre de Grévy,https://www.monde-animal.fr/fiche_animaux/zebre-de-grevy/
389,Zèbre de montagne,https://www.monde-animal.fr/fiche_animaux/zebre-de-montagne/


In [169]:
df_animals_bis_bis = df_animals_bis_bis.drop(df_animals_bis_bis.index[:36])
df_animals_bis_bis

,name,Links
36,Agame aquatique,https://www.monde-animal.fr/fiche_animaux/dragon-deau-australien/
37,Agame barbu,https://www.monde-animal.fr/fiche_animaux/dragon-barbu/
38,Aigle à tête blanche,https://www.monde-animal.fr/fiche_animaux/pygargue-a-tete-blanche/
39,Aigle australien,https://www.monde-animal.fr/fiche_animaux/aigle-australien/
40,Aigle des Philippines,https://www.monde-animal.fr/aigle-des-philippines/
...,...,...
386,Wapiti,https://www.monde-animal.fr/fiche_animaux/wapiti/
387,Wombat commun,https://www.monde-animal.fr/fiche_animaux/wombat-commun/
388,Zèbre de Grévy,https://www.monde-animal.fr/fiche_animaux/zebre-de-grevy/
389,Zèbre de montagne,https://www.monde-animal.fr/fiche_animaux/zebre-de-montagne/


In [170]:
df_animals_bis_bis.reset_index(drop=True,inplace=True)
df_animals_bis_bis


,name,Links
0,Agame aquatique,https://www.monde-animal.fr/fiche_animaux/dragon-deau-australien/
1,Agame barbu,https://www.monde-animal.fr/fiche_animaux/dragon-barbu/
2,Aigle à tête blanche,https://www.monde-animal.fr/fiche_animaux/pygargue-a-tete-blanche/
3,Aigle australien,https://www.monde-animal.fr/fiche_animaux/aigle-australien/
4,Aigle des Philippines,https://www.monde-animal.fr/aigle-des-philippines/
...,...,...
350,Wapiti,https://www.monde-animal.fr/fiche_animaux/wapiti/
351,Wombat commun,https://www.monde-animal.fr/fiche_animaux/wombat-commun/
352,Zèbre de Grévy,https://www.monde-animal.fr/fiche_animaux/zebre-de-grevy/
353,Zèbre de montagne,https://www.monde-animal.fr/fiche_animaux/zebre-de-montagne/


In [171]:
df_animals_bis_bis["name"]=df_animals_bis_bis["name"].apply(clean_name)
df_animals_bis_bis

,name,Links
0,agame aquatique,https://www.monde-animal.fr/fiche_animaux/dragon-deau-australien/
1,agame barbu,https://www.monde-animal.fr/fiche_animaux/dragon-barbu/
2,aigle a tete blanche,https://www.monde-animal.fr/fiche_animaux/pygargue-a-tete-blanche/
3,aigle australien,https://www.monde-animal.fr/fiche_animaux/aigle-australien/
4,aigle des philippines,https://www.monde-animal.fr/aigle-des-philippines/
...,...,...
350,wapiti,https://www.monde-animal.fr/fiche_animaux/wapiti/
351,wombat commun,https://www.monde-animal.fr/fiche_animaux/wombat-commun/
352,zebre de grevy,https://www.monde-animal.fr/fiche_animaux/zebre-de-grevy/
353,zebre de montagne,https://www.monde-animal.fr/fiche_animaux/zebre-de-montagne/


In [172]:
extracted_data = []

for index, row in df_animals_bis_bis.iterrows():
    animal_link = row['Links']
    response = requests.get(animal_link)
    soup = BeautifulSoup(response.content, 'html.parser')
    info_div = soup.find('div', class_="elementor-element elementor-element-5e65e78 elementor-widget elementor-widget-text-editor")
    
    # Extraire les informations si le div est trouvé
    if info_div:
        ul = info_div.find('ul')
        if ul:
            info_list = ul.find_all('li')
            animal_info = {item.text.split(':')[0].strip(): item.text.split(':')[1].strip() for item in info_list}
            animal_info['Name'] = row['name']  # Ajouter le nom de l'animal
            extracted_data.append(animal_info)

df_extracted_info = pd.DataFrame(extracted_data)
df_extracted_info


,Règne,Embranchement/sous-embranchement,Classe,Ordre,Famille,Genre,Espèce,Nom scientifique,Name,Famille/sous-famille,Ordre/Infra-Ordre,Sous-famille,Sous-ordre,Super-famille,Classe/infra-classe,Infra-ordre,Sous-espèce
0,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Intellagama,lesueurii,Intellagama lesueurii,agame aquatique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Pogona,vitticeps,Pogona vitticeps,agame barbu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Haliaeetus,leucocephalus,Haliaeetus leucocephalus,aigle a tete blanche,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Aquila,audax,Aquila audax,aigle australien,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridae,Harpia,harpya,Harpia harpya,aigle harpie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341,Animal,Chordé vertébré,Mammifère placentaire,Artiodactyle ruminant,Cervidae cervinae,Cervus,canadensis,Cervus canadensis,wapiti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,Animal,Chordé vertébré,Mammifère marsupial,Diprotodonte,Vombatidés,Vombatus,Ursinus,Vombatus ursinus,wombat commun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
343,Animal,Chordé vertébré,Mammifère placentaire,Périssodactyle ruminant,Equidé,Equus,grevyi,Equus grevyi,zebre de grevy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,Animal,Chordé vertébré,Mammifère placentaire,Périssodactyle ruminant,Equidae,Equus,zebra,Equus zebra,zebre de montagne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


certains animaux ne possedent pas la meme cle d'image. je peux scrapper que les images d'un seul site car ca allait se superposer lors du merge des infos des deux sites

In [173]:
from IPython.display import Image, display

animal_links_list=[]
image_animaux= []
for index, row in df_animals_bis_bis.iterrows():
    animal_link = row['Links']
    animal_links_list.append(animal_link)

session = requests.Session()

for animal_link in animal_links_list:
    if len(image_animaux) >= 346:  # Vérifie si la longueur désirée est atteinte
        break
    try:
        response = session.get(animal_link)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Trouver le div contenant l'image
        img_div = soup.find('div', class_="elementor-element elementor-element-4683d27 elementor-widget elementor-widget-image")
        
        if img_div and img_div.find('img'):
            img_tag = img_div.find('img')
            image_url = img_tag.get('data-src')
            image_animaux.append(image_url)
        else:
            print("Image non trouvée pour cet animal.")
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête : {e}")
df_extracted_info['Image_URL'] = image_animaux

Image non trouvée pour cet animal.
Image non trouvée pour cet animal.
Image non trouvée pour cet animal.
Image non trouvée pour cet animal.
Image non trouvée pour cet animal.


In [174]:
extracted_data = []
animal_links_list=[]


for index, row in df_animals_bis_bis.iterrows():
    animal_link = row['Links']
    animal_links_list.append(animal_link)

session = requests.Session()

for animal_link in animal_links_list:
    if len(extracted_data) >= 346:  
        break
    try:
        time.sleep(1)
        response = session.get(animal_link)
        soup = BeautifulSoup(response.content, 'html.parser')

        div_container = soup.find('div', class_="elementor-element elementor-element-dd446b5 elementor-widget elementor-widget-text-editor")
        if div_container:
            
            strong_tag = div_container.find('strong') 
            if strong_tag:
                strong_text = strong_tag.get_text() 
            else:
                strong_text="Texte non trouvé"
        else :
            strong_text="Texte non trouvé"
        extracted_data.append(strong_text)


    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête : {e}")
df_extracted_info['Zone']=extracted_data

probleme de python donc streamlit complique.

In [175]:
colonnes_a_verifier = ['Famille/sous-famille', 'Ordre/Infra-Ordre', 'Sous-famille', 'Sous-ordre', 'Super-famille', 'Classe/infra-classe', 'Infra-ordre', 'Sous-espèce']

# Suppression des colonnes spécifiées si elles ne contiennent que des NaN
df_cleaned = df_extracted_info.drop(colonnes_a_verifier, axis=1)


In [176]:
df_cleaned["Nom scientifique"]=df_cleaned["Nom scientifique"].apply(clean_name)
df_cleaned = df_cleaned.rename(columns={'Nom scientifique': 'Nom latin'})


In [177]:
df_cleaned=df_cleaned.rename(columns={'Name': 'name','Embranchement/sous-embranchement':'Embranchement'})

In [178]:
df_cleaned

,Règne,Embranchement,Classe,Ordre,Famille,Genre,Espèce,Nom latin,name,Image_URL,Zone
0,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Intellagama,lesueurii,intellagama lesueurii,agame aquatique,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-dragon-d-eau-australien.jpg,Océanie.
1,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Pogona,vitticeps,pogona vitticeps,agame barbu,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-dragon-barbu-central-1.jpg,Océanie
2,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Haliaeetus,leucocephalus,haliaeetus leucocephalus,aigle a tete blanche,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-pygargue-a-tete-blanche.jpg,Amérique du Nord
3,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Aquila,audax,aquila audax,aigle australien,https://www.monde-animal.fr/wp-content/uploads/2020/07/fiche-animale-monde-animal-aigle-australien-d-australie.jpg,Texte non trouvé
4,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridae,Harpia,harpya,harpia harpya,aigle harpie,https://www.monde-animal.fr/wp-content/uploads/2020/10/fiche-animale-monde-animal-harpie-feroce.jpg,Texte non trouvé
...,...,...,...,...,...,...,...,...,...,...,...
341,Animal,Chordé vertébré,Mammifère placentaire,Artiodactyle ruminant,Cervidae cervinae,Cervus,canadensis,cervus canadensis,wapiti,https://www.monde-animal.fr/wp-content/uploads/2020/06/fiche-animale-monde-animal-vigogne.jpg,Afrique
342,Animal,Chordé vertébré,Mammifère marsupial,Diprotodonte,Vombatidés,Vombatus,Ursinus,vombatus ursinus,wombat commun,https://www.monde-animal.fr/wp-content/uploads/2020/07/fiche-animale-monde-animal-wallaby-de-bennett.jpg,Australie continentale
343,Animal,Chordé vertébré,Mammifère placentaire,Périssodactyle ruminant,Equidé,Equus,grevyi,equus grevyi,zebre de grevy,https://www.monde-animal.fr/wp-content/uploads/2020/07/fiche-animale-monde-animal-wallaby-de-bennett.jpg,l’île de Madagascar
344,Animal,Chordé vertébré,Mammifère placentaire,Périssodactyle ruminant,Equidae,Equus,zebra,equus zebra,zebre de montagne,https://www.monde-animal.fr/wp-content/uploads/2020/06/fiche-animale-wallaby-de-parma.jpg,’île de Madagascar


# Dataset Merge

In [179]:
# Concaténation des deux dataframes
df_concatene = pd.concat([df_cleaned, df_animal_info])
df_final = df_concatene.reset_index(drop=True)

In [180]:
df_final.head(15)

,Règne,Embranchement,Classe,Ordre,Famille,Genre,Espèce,Nom latin,name,Image_URL,Zone
0,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Intellagama,lesueurii,intellagama lesueurii,agame aquatique,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-dragon-d-eau-australien.jpg,Océanie.
1,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Pogona,vitticeps,pogona vitticeps,agame barbu,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-dragon-barbu-central-1.jpg,Océanie
2,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Haliaeetus,leucocephalus,haliaeetus leucocephalus,aigle a tete blanche,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-pygargue-a-tete-blanche.jpg,Amérique du Nord
3,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Aquila,audax,aquila audax,aigle australien,https://www.monde-animal.fr/wp-content/uploads/2020/07/fiche-animale-monde-animal-aigle-australien-d-australie.jpg,Texte non trouvé
4,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridae,Harpia,harpya,harpia harpya,aigle harpie,https://www.monde-animal.fr/wp-content/uploads/2020/10/fiche-animale-monde-animal-harpie-feroce.jpg,Texte non trouvé
5,Animal,Chordé vertébré,Reptilia,Crocodilia,NaN,Alligator,mississippiensis,alligator mississippiensis,alligator,https://www.monde-animal.fr/wp-content/uploads/2020/12/fiche-animale-monde-animal-alligator-d-amerique-mississipiensis.jpg,Amérique du Sud
6,Animal,Chordé vertébré,Reptilia,Crocodilia,NaN,Alligator,mississippiensis,alligator mississippiensis,alligator d'amerique,https://www.monde-animal.fr/wp-content/uploads/2020/12/fiche-animale-monde-animal-alligator-d-amerique-mississipiensis.jpg,Texte non trouvé
7,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,maculatum,ambystoma maculatum,ambystome macule,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-maculée.jpg,Amérique du Nord
8,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,opacum,ambystoma opacum,ambystome marbre,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-marbrée.jpg,Amérique du Nord
9,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,tigrinum,ambystoma tigrinum,ambystome tigre,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-tigree.jpg,Amérique du Nord


In [181]:
df_sans_doublons = df_concatene.drop_duplicates(subset='Nom latin')
df_sans_doublons.head(20)

,Règne,Embranchement,Classe,Ordre,Famille,Genre,Espèce,Nom latin,name,Image_URL,Zone
0,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Intellagama,lesueurii,intellagama lesueurii,agame aquatique,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-dragon-d-eau-australien.jpg,Océanie.
1,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Pogona,vitticeps,pogona vitticeps,agame barbu,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-dragon-barbu-central-1.jpg,Océanie
2,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Haliaeetus,leucocephalus,haliaeetus leucocephalus,aigle a tete blanche,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-pygargue-a-tete-blanche.jpg,Amérique du Nord
3,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Aquila,audax,aquila audax,aigle australien,https://www.monde-animal.fr/wp-content/uploads/2020/07/fiche-animale-monde-animal-aigle-australien-d-australie.jpg,Texte non trouvé
4,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridae,Harpia,harpya,harpia harpya,aigle harpie,https://www.monde-animal.fr/wp-content/uploads/2020/10/fiche-animale-monde-animal-harpie-feroce.jpg,Texte non trouvé
5,Animal,Chordé vertébré,Reptilia,Crocodilia,NaN,Alligator,mississippiensis,alligator mississippiensis,alligator,https://www.monde-animal.fr/wp-content/uploads/2020/12/fiche-animale-monde-animal-alligator-d-amerique-mississipiensis.jpg,Amérique du Sud
7,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,maculatum,ambystoma maculatum,ambystome macule,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-maculée.jpg,Amérique du Nord
8,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,opacum,ambystoma opacum,ambystome marbre,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-marbrée.jpg,Amérique du Nord
9,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,tigrinum,ambystoma tigrinum,ambystome tigre,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-tigree.jpg,Amérique du Nord
10,Animal,Chordé vertébré,Mammifère placentaire,Cétartiodactyle,Antilocapridé,Antilocapra,americana,antilocapra americana,antilocapre,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-pronghorn.jpg,Amérique du Nord


Les infos sur la localisation des animaux ne sont disponibles que sur le second site.

### IUCN

In [120]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [121]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://www.iucnredlist.org/search/list")

C:\Users\valen\AppData\Local\Temp\ipykernel_26912\447631463.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Long scrapping car on doit aller chercher l'etat de chaque animaux un par un. 

In [122]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

search_bar = driver.find_element(By.CLASS_NAME, "search--site")
search_bar.send_keys("intellagama lesueurii")  
search_bar.send_keys(Keys.RETURN)

time.sleep(5)

wait = WebDriverWait(driver, 10)

species_link=driver.find_element(By.CSS_SELECTOR, "a.link--faux")
species_link.click()

time.sleep(5)

conservation_status_element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#assessment-information strong")))
    
conservation_status = conservation_status_element.text
print(conservation_status)


Least Concern


on doit ajouter un try car on a remarrque que les noms etait different dans la bdd et la recherche du site a une ou deux lettre pres.

In [127]:
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome(ChromeDriverManager().install())
 
extracted_data = []

for index, row in df_sans_doublons.iterrows():
    latin_name = row['Nom latin']  

    driver.get("https://www.iucnredlist.org/search/list")

    search_bar = driver.find_element(By.CLASS_NAME, "search--site")
    search_bar.send_keys(latin_name)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(3)
    try:
        species_link = driver.find_element(By.CSS_SELECTOR, "a.link--faux")
        species_link.click()
    except NoSuchElementException:
        print(f"Aucun résultat trouvé pour {latin_name}")
        conservation_status = 'Aucun résultat'
        extracted_data.append({'Nom latin': latin_name, 'Statut de conservation': conservation_status})
        continue  

    time.sleep(2)

    conservation_status_element = driver.find_element(By.CSS_SELECTOR, "#assessment-information strong")
    conservation_status = conservation_status_element.text

    # Ajouter les informations extraites à la liste
    extracted_data.append({'Nom latin': latin_name, 'Statut de conservation': conservation_status})

driver.quit()

C:\Users\valen\AppData\Local\Temp\ipykernel_26912\2351654894.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Aucun résultat trouvé pour harpia harpya
Aucun résultat trouvé pour alligator mississippiensis
Aucun résultat trouvé pour antilope cervicapra
Aucun résultat trouvé pour papio papio
Aucun résultat trouvé pour canis lupus dingo
Aucun résultat trouvé pour camelus dromedarius
Aucun résultat trouvé pour martes foina
Aucun résultat trouvé pour bos frontalis
Aucun résultat trouvé pour hippopotamus amphibius
Aucun résultat trouvé pour canis lupus arctos
Aucun résultat trouvé pour ranoidea caerulea / dryopsophus caeruleus 
Aucun résultat trouvé pour vicugna pacos
Aucun résultat trouvé pour bubalus bubalis
Aucun résultat trouvé pour camelus bactrianus ferus
Aucun résultat trouvé pour canis lupus hallstromi
Aucun résultat trouvé pour coloeus monedula
Aucun résultat trouvé pour cygnus atratus
Aucun résultat trouvé pour taurotragus derbianus
Aucun résultat trouvé pour taurotragus oryx
Aucun résultat trouvé pour mustela putorius furo
Aucun résultat trouvé pour ursus arctos horribilis
Aucun résultat 

on a du mettre un temps rapide entre les recherches sinon la compilation serait bcp trop longues. Ca a cause aucun resultat pour des animaux qui avait bien le mot 

In [128]:
extracted_data

[{'Nom latin': 'intellagama lesueurii',
  'Statut de conservation': 'Least Concern'},
 {'Nom latin': 'pogona vitticeps', 'Statut de conservation': 'Least Concern'},
 {'Nom latin': 'haliaeetus leucocephalus',
  'Statut de conservation': 'Least Concern'},
 {'Nom latin': 'aquila audax', 'Statut de conservation': 'Least Concern'},
 {'Nom latin': 'harpia harpya', 'Statut de conservation': 'Aucun résultat'},
 {'Nom latin': 'alligator mississippiensis',
  'Statut de conservation': 'Aucun résultat'},
 {'Nom latin': 'ambystoma maculatum',
  'Statut de conservation': 'Least Concern'},
 {'Nom latin': 'ambystoma opacum', 'Statut de conservation': 'Least Concern'},
 {'Nom latin': 'ambystoma tigrinum',
  'Statut de conservation': 'Least Concern'},
 {'Nom latin': 'antilocapra americana',
  'Statut de conservation': 'Least Concern'},
 {'Nom latin': 'antilope cervicapra',
  'Statut de conservation': 'Aucun résultat'},
 {'Nom latin': 'saiga tatarica', 'Statut de conservation': 'Near Threatened'},
 {'Nom

In [130]:
liste_Vulnerability=pd.DataFrame(extracted_data)
chemin_fichier = r'C:\Users\valen\Documents\PORTABLE TD A5S1\Webscrap.csv'
liste_Vulnerability.to_csv(chemin_fichier, index=False)

In [131]:
liste_Vulnerability

,Nom latin,Statut de conservation
0,intellagama lesueurii,Least Concern
1,pogona vitticeps,Least Concern
2,haliaeetus leucocephalus,Least Concern
3,aquila audax,Least Concern
4,harpia harpya,Aucun résultat
...,...,...
487,chloris chloris,Least Concern
488,vipera aspis,Least Concern
489,mustela lutreola,Critically Endangered
490,bos grunniens,Aucun résultat


In [182]:
df_sans_doublons["Vulnerability"]=liste_Vulnerability['Statut de conservation']

C:\Users\valen\AppData\Local\Temp\ipykernel_26912\1377446133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sans_doublons["Vulnerability"]=liste_Vulnerability['Statut de conservation']


In [183]:
dffinal=df_sans_doublons
dffinal.head(30)

,Règne,Embranchement,Classe,Ordre,Famille,Genre,Espèce,Nom latin,name,Image_URL,Zone,Vulnerability
0,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Intellagama,lesueurii,intellagama lesueurii,agame aquatique,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-dragon-d-eau-australien.jpg,Océanie.,Least Concern
1,Animal,Chordé vertébré,Reptile,Squamate,Agamidae,Pogona,vitticeps,pogona vitticeps,agame barbu,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-dragon-barbu-central-1.jpg,Océanie,Least Concern
2,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Haliaeetus,leucocephalus,haliaeetus leucocephalus,aigle a tete blanche,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-pygargue-a-tete-blanche.jpg,Amérique du Nord,Least Concern
3,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridé,Aquila,audax,aquila audax,aigle australien,https://www.monde-animal.fr/wp-content/uploads/2020/07/fiche-animale-monde-animal-aigle-australien-d-australie.jpg,Texte non trouvé,Least Concern
4,Animal,Chordé vertébré,Oiseaux,Accipitriforme,Accipitridae,Harpia,harpya,harpia harpya,aigle harpie,https://www.monde-animal.fr/wp-content/uploads/2020/10/fiche-animale-monde-animal-harpie-feroce.jpg,Texte non trouvé,Aucun résultat
5,Animal,Chordé vertébré,Reptilia,Crocodilia,NaN,Alligator,mississippiensis,alligator mississippiensis,alligator,https://www.monde-animal.fr/wp-content/uploads/2020/12/fiche-animale-monde-animal-alligator-d-amerique-mississipiensis.jpg,Amérique du Sud,Aucun résultat
7,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,maculatum,ambystoma maculatum,ambystome macule,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-maculée.jpg,Amérique du Nord,Least Concern
8,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,opacum,ambystoma opacum,ambystome marbre,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-marbrée.jpg,Amérique du Nord,Least Concern
9,Animal,Chordé vertébré,Amphibien,Caudata,Ambystomatidae,Ambystoma,tigrinum,ambystoma tigrinum,ambystome tigre,https://www.monde-animal.fr/wp-content/uploads/2020/11/fiche-animale-monde-animal-salamandre-tigree.jpg,Amérique du Nord,Least Concern
10,Animal,Chordé vertébré,Mammifère placentaire,Cétartiodactyle,Antilocapridé,Antilocapra,americana,antilocapra americana,antilocapre,https://www.monde-animal.fr/wp-content/uploads/2020/09/fiche-animale-monde-animal-pronghorn.jpg,Amérique du Nord,Aucun résultat


In [184]:
chemin_fichier = r'C:\Users\valen\Documents\PORTABLE TD A5S1\Webscrap\datasetfinal.csv'
dffinal.to_csv(chemin_fichier, index=False)